In [13]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, T5Tokenizer, T5ForConditionalGeneration

from datasets import *
import numpy as np

import torch
import ast


SEED = 42

### Load tokenizer and model

In [14]:
model_name = "mrm8488/t5-small-finetuned-squadv2" # small model
# model_name = "mrm8488/t5-base-finetuned-squadv2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


### Load the dataset

In [15]:
# Load the dataset from file and split it into train and test datasets
data = load_dataset('csv', data_files="../../data/clean/sustainability-report-2020-squad-format.csv", delimiter=";", split='train').train_test_split(test_size=0.3, shuffle=True, seed=SEED)

Found cached dataset csv (C:/Users/Luka/.cache/huggingface/datasets/csv/default-d8382661cd597e83/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached split indices for dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-d8382661cd597e83\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-c261d5613d28d856.arrow and C:\Users\Luka\.cache\huggingface\datasets\csv\default-d8382661cd597e83\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-e61829c1e4a24b65.arrow


In [16]:
# Reformat the train and test set such as they adhere to the SQuAD format (reading from cvs loads strings not objects as expected)
data["test"] = data["test"].map(lambda example: ast.literal_eval(example["answers"]))
data["test"] = data["test"].map(lambda example: {"question": example["question"], "context": example["context"], "answers": {"text": example["text"], "answer_start": example["answer_start"]}})
data["test"] = data["test"].remove_columns(["text", "answer_start"])

data["train"] = data["train"].map(lambda example: ast.literal_eval(example["answers"]))
data["train"] = data["train"].map(lambda example: {"question": example["question"], "context": example["context"], "answers": {"text": example["text"], "answer_start": example["answer_start"]}})
data["train"] = data["train"].remove_columns(["text", "answer_start"])

Loading cached processed dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-d8382661cd597e83\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-0b15501cefb41ff7.arrow
Loading cached processed dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-d8382661cd597e83\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-6c4455904f60e079.arrow
Loading cached processed dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-d8382661cd597e83\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-65eb14b3b79cbed9.arrow
Loading cached processed dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-d8382661cd597e83\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-559c811f459458f4.arrow


In [17]:
data["train"][0]

{'question': 'How does the Bank prevent or manage cyber risks arising from the increased volume of work from home?',
 'context': 'nages cyber risks arising from the increased volumeof work from home through implemented measures, some of which areexplained below, namelyPromoting Freedom of Expr',
 'answers': {'answer_start': [69], 'text': ['through implemented measures']},
 'id': 132}

### Tokenize the dataset

In [18]:
# process the examples in input and target text format and the eos token at the end 
def add_eos_to_examples(example):
    example['input_text'] = 'question: %s  context: %s </s>' % (example['question'], example['context'])
    example['target_text'] = '%s </s>' % example['answers']['text'][0]
    return example

# tokenize the examples
def convert_to_features(examples):
    model_inputs = tokenizer(examples['input_text'], pad_to_max_length=True, max_length=512)
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['target_text'], pad_to_max_length=True, max_length=16) # if max length is high eg. 128 the model starts to output only <pad> tokens
        labels[labels['input_ids'] == tokenizer.pad_token_id] = -100

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [19]:
train_data, test_data = data["train"], data["test"]

train_data = train_data.map(add_eos_to_examples, load_from_cache_file=False)
train_data = train_data.map(convert_to_features, batched=True, load_from_cache_file=False)

test_data = test_data.map(add_eos_to_examples, load_from_cache_file=False)
test_data = test_data.map(convert_to_features, batched=True, load_from_cache_file=False)

Map:   0%|          | 0/129 [00:00<?, ? examples/s]

Map:   0%|          | 0/129 [00:00<?, ? examples/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Users\Luka\miniconda3\envs\project_ds\lib\site-packages\transformers\tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
c:\Users\Luka\miniconda3\envs\project_ds\lib\site-packages\transformers\tokeniza

Map:   0%|          | 0/56 [00:00<?, ? examples/s]

Map:   0%|          | 0/56 [00:00<?, ? examples/s]

In [20]:
train_data[0]

{'question': 'How does the Bank prevent or manage cyber risks arising from the increased volume of work from home?',
 'context': 'nages cyber risks arising from the increased volumeof work from home through implemented measures, some of which areexplained below, namelyPromoting Freedom of Expr',
 'answers': {'answer_start': [69], 'text': ['through implemented measures']},
 'id': 132,
 'input_text': 'question: How does the Bank prevent or manage cyber risks arising from the increased volume of work from home?  context: nages cyber risks arising from the increased volumeof work from home through implemented measures, some of which areexplained below, namelyPromoting Freedom of Expr </s>',
 'target_text': 'through implemented measures </s>',
 'input_ids': [822,
  10,
  571,
  405,
  8,
  1925,
  1709,
  42,
  1865,
  9738,
  5217,
  3,
  14739,
  45,
  8,
  1936,
  2908,
  13,
  161,
  45,
  234,
  58,
  2625,
  10,
  3,
  9761,
  7,
  9738,
  5217,
  3,
  14739,
  45,
  8,
  1936,
  2908

### Fine tunning

In [21]:
import numpy as np
import torch
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments


name = model_name.split("/")[-1]
training_args = Seq2SeqTrainingArguments(
    output_dir = f"./models",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [22]:
trainer.train()
trainer.save_model(f"./models/{name}-finetuned-NLB-QA")

  0%|          | 0/170 [00:00<?, ?it/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 12.92862606048584, 'eval_runtime': 0.342, 'eval_samples_per_second': 163.759, 'eval_steps_per_second': 20.47, 'epoch': 1.0}


  0%|          | 0/7 [00:00<?, ?it/s]

...

{'eval_loss': 1.044721245765686, 'eval_runtime': 0.3001, 'eval_samples_per_second': 186.628, 'eval_steps_per_second': 23.329, 'epoch': 8.0}


  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.9810191988945007, 'eval_runtime': 0.2862, 'eval_samples_per_second': 195.646, 'eval_steps_per_second': 24.456, 'epoch': 9.0}


  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.959847092628479, 'eval_runtime': 0.3071, 'eval_samples_per_second': 182.333, 'eval_steps_per_second': 22.792, 'epoch': 10.0}
{'train_runtime': 29.3392, 'train_samples_per_second': 43.968, 'train_steps_per_second': 5.794, 'train_loss': 4.664511646943934, 'epoch': 10.0}


In [23]:
def get_answer(question, context):
    input_text = "question: %s  context: %s" % (question, context)
    features = tokenizer([input_text], return_tensors='pt')

    output = model.generate(input_ids=features['input_ids'], attention_mask=features['attention_mask'])

    return tokenizer.decode(output[0])

In [24]:
question = test_data[0]["question"]
context = test_data[0]["context"]
answer = test_data[0]["answers"]["text"][0]
print(f"Question: {question} \nContext: {context} \nAnswer: {answer}")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
print(f"Squad model answer: {get_answer(question, context)}")

tokenizer = AutoTokenizer.from_pretrained(f"./models/{name}-finetuned-NLB-QA", local_files_only=True)
model = AutoModelForSeq2SeqLM.from_pretrained(f"./models/{name}-finetuned-NLB-QA", local_files_only=True)
print(f"Our model answer: {get_answer(question, context)}")

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


Question: How many employees departed from NLB Group in 2020? 
Context: NLB Group In total, 382 employeesdeparted from NLB Group in 2020.In total, 162 employees de 
Answer: 382


The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


Squad model answer: <pad> 382</s>
Our model answer: <pad> 382</s>


In [30]:
answers = [temp["answers"]["text"][0] for temp in test_data]

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
squad_answers = [get_answer(question, context) for question, context in zip(test_data["question"], test_data["context"])]

tokenizer = AutoTokenizer.from_pretrained(f"./models/{name}-finetuned-NLB-QA", local_files_only=True)
model = AutoModelForSeq2SeqLM.from_pretrained(f"./models/{name}-finetuned-NLB-QA", local_files_only=True)
our_answers = [get_answer(question, context) for question, context in zip(test_data["question"], test_data["context"])]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
c:\Users\Luka\miniconda3\envs\project_ds\lib\site-packages\transformers\generation\utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [31]:
squad_answers

['<pad> 382</s>',
 '<pad>green/sustainability financing</s>',
 '<pad> 1</s>',
 '<pad> 2021</s>',
 '<pad> law, collectiveagreements and internal regulations</s>',
 '<pad> their Social and EnvironmentalPolicy</s>',
 '<pad> equal opportunities, justice, and promotingculture focused on performance</s>',
 '<pad> introduction to kindergarten</s>',
 '<pad> None</s>',
 '<pad> three</s>',
 '<pad> 30th December 2020</s>',
 '<pad> 2020</s>',
 '<pad> 6.7%</s>',
 '<pad>Komercijalna Banka a.d. Beograd KB',
 '<pad> 2020</s>',
 '<pad> 2021</s>',
 '<pad> sculptorBogdan Darmanovi<unk></s>',
 '<pad> 4,769</s>',
 '<pad> World Institute forSustainability and Ethics in Rising Economies</s>',
 '<pad> 2018</s>',
 '<pad> 2020</s>',
 '<pad> 23 million EUR</s>',
 '<pad> 307</s>',
 '<pad> higher quality of life of the wider society</s>',
 '<pad> EUR 340 million</s>',
 '<pad> Retail Banking in Slovenia, Corporate Bankingin Slovenia, and Strategic Foreign Markets</s>',
 '<pad> 30.12.2020</s>',
 '<pad> None</s>',
 '

In [32]:
our_answers

['<pad> 382</s>',
 '<pad>green/sustainability financing</s>',
 '<pad> 1</s>',
 '<pad> 2021</s>',
 '<pad> law, collectiveagreements and internal regulations</s>',
 '<pad> Social and EnvironmentalPolicy</s>',
 '<pad> equal opportunities, justice, and promotingculture focused on performance</s>',
 '<pad> introduction to kindergarten</s>',
 '<pad> None</s>',
 '<pad> three</s>',
 '<pad> 30th December 2020</s>',
 '<pad> 2020</s>',
 '<pad> 6.7%</s>',
 '<pad>Komercijalna Banka a.d. Beograd KB',
 '<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 '<pad> 2021</s>',
 '<pad> sculptorBogdan Darmanovi<unk></s>',
 '<pad> 4,769</s>',
 '<pad> World Institute forSustainability and Ethics in Rising Economies</s>',
 '<pad> 2018</s>',
 '<pad> 2020</s>',
 '<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 '<pad> 307</s>',
 '<pad> a higher quality of life of the wider society</s>',
 '<pad><pad><pad><pa

In [33]:
import evaluate
bertscore = evaluate.load("bertscore")

results = bertscore.compute(predictions=squad_answers, references=answers, lang="en")
# Embeddings bases evaluation
print(f"Squad\nF1: {np.array(results['f1']).mean()}, Precision: {np.array(results['precision']).mean()}, Recall: {np.array(results['recall']).mean()}")

results = bertscore.compute(predictions=our_answers, references=answers, lang="en")
# Embeddings bases evaluation
print(f"Our model\nF1: {np.array(results['f1']).mean()}, Precision: {np.array(results['precision']).mean()}, Recall: {np.array(results['recall']).mean()}")

c:\Users\Luka\miniconda3\envs\project_ds\lib\site-packages\huggingface_hub-0.13.3-py3.8.egg\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Luka\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development


Squad
F1: 0.8861216157674789, Precision: 0.8740572429129055, Recall: 0.8988280573061534
Our model
F1: 0.8790042740958077, Precision: 0.8653222109590258, Recall: 0.8937693855592183
